In [1]:
import requests as rq
import pandas as pd

In [ ]:
# base_url = 'https://swapi.dev/api/people/'

In [2]:
# data = rq.get(f'{base_url}10/').json()
# character_fields = data.keys()
# print(*character_fields, sep='\n')

In [3]:
# rq.get(f'{base_url}10/').json()

In [22]:
def get_character(id):
    base_url = 'https://swapi.dev/api/people'
    # get the url
    response = rq.get(f'{base_url}/{id}')

    # success
    if response.status_code == 200:
        content = response.json()
    elif response.status_code == 404:
        print(f'{base_url}/{id} not found!')
        return

    
    related_fields = ['homeworld', 'films', 'species', 'vehicles', 'starships']

    for field in related_fields:
        id_values = []
        try:
            
            if content[field]:  # if the field is not empty
                # parse the links from films, vehicles and starships
                if field != 'homeworld':  
                    for link in content[field]:
                        # parse the id value in the link    
                        id_values.append(int(link.split('/')[-2]))
                    # add the id values into the corresponding field key
                    content[field] = id_values
                
                # parse the homeworld (just a single string value)
                else:
                    # get the homeworld id
                    content['homeworld'] = int(content['homeworld'].split('/')[-2])
            
            # parse species field
            # in case of human characters, the species field is an empty list
            elif field == 'species' and not content[field]:
                content[field] = 'human'
        
        except:
            print(f"{field} doesn't exist in character url: {base_url}/{id}")
            # if there's a missing field, don't consume the API link for this character
            break
             
    # remove created and edited fields
    try:
        del(content['created'])
        del(content['edited'])
    except:
        pass
    
    return content

Scrapping takes 

In [ ]:
while True:
    

In [ ]:
# a list of dictionaries with the character info
character_list = []
id = 1
while True:
    # the character page with id 17 doesn't exist!
    if id == 17:
        id += 1
        pass
    char = get_character(id)
    if char:
        character_list.append(char)
        id += 1
    else:
        print(f'\nInformation from {id - 1} characters has been extracted.')
        break

https://swapi.dev/api/people/84 not found!
Information from 83 characters has been extracted.


In [27]:
character_list[:2]

[{'name': 'Luke Skywalker',
  'height': '172',
  'mass': '77',
  'hair_color': 'blond',
  'skin_color': 'fair',
  'eye_color': 'blue',
  'birth_year': '19BBY',
  'gender': 'male',
  'homeworld': 1,
  'films': [1, 2, 3, 6],
  'species': 'human',
  'vehicles': [14, 30],
  'starships': [12, 22],
  'url': 'https://swapi.dev/api/people/1/'},
 {'name': 'C-3PO',
  'height': '167',
  'mass': '75',
  'hair_color': 'n/a',
  'skin_color': 'gold',
  'eye_color': 'yellow',
  'birth_year': '112BBY',
  'gender': 'n/a',
  'homeworld': 1,
  'films': [1, 2, 3, 4, 5, 6],
  'species': [2],
  'vehicles': [],
  'starships': [],
  'url': 'https://swapi.dev/api/people/2/'}]

In [ ]:
{
    "name": "unknown", 
    "rotation_period": "0", 
    "orbital_period": "0", 
    "diameter": "0", 
    "climate": "unknown", 
    "gravity": "unknown", 
    "terrain": "unknown", 
    "surface_water": "unknown", 
    "population": "unknown", 
    "residents": [
        "https://swapi.dev/api/people/20/", 
        "https://swapi.dev/api/people/23/", 
        "https://swapi.dev/api/people/29/", 
        "https://swapi.dev/api/people/32/", 
        "https://swapi.dev/api/people/75/"
    ], 
    "films": [], 
    "created": "2014-12-15T12:25:59.569000Z", 
    "edited": "2014-12-20T20:58:18.466000Z", 
    "url": "https://swapi.dev/api/planets/28/"
}

In [35]:
def get_planet(id):
    base_url = 'https://swapi.dev/api/planets'
    # get the url
    response = rq.get(f'{base_url}/{id}')

    # success
    if response.status_code == 200:
        content = response.json()
    elif response.status_code == 404:
        print(f'{base_url}/{id} not found!')
        return

    if content['name'] == 'unknown':
        return
    
    for field in ['films', 'residents']:
        # if the field is not empty
        if content[field]:
            id_values = []
            for link in content[field]:
                # parse the id value in the link    
                id_values.append(int(link.split('/')[-2]))
            content[field] = id_values
             
    # remove created and edited fields
    try:
        del(content['created'])
        del(content['edited'])
    except:
        pass
    
    return content

In [36]:
# a list of dictionaries with the character info
planets_list = []
id = 1
while True:
    # the character page with id 17 doesn't exist!
    # if id == 17:
    #     id += 1
    #     pass
    planet = get_planet(id)
    if planet:
        planets_list.append(planet)
        id += 1
    else:
        print(f'\nInformation from {id} planets has been extracted.')
        break


Information from 28 planets has been extracted.


In [38]:
planets_list[:2]

[{'name': 'Tatooine',
  'rotation_period': '23',
  'orbital_period': '304',
  'diameter': '10465',
  'climate': 'arid',
  'gravity': '1 standard',
  'terrain': 'desert',
  'surface_water': '1',
  'population': '200000',
  'residents': [1, 2, 4, 6, 7, 8, 9, 11, 43, 62],
  'films': [1, 3, 4, 5, 6],
  'url': 'https://swapi.dev/api/planets/1/'},
 {'name': 'Alderaan',
  'rotation_period': '24',
  'orbital_period': '364',
  'diameter': '12500',
  'climate': 'temperate',
  'gravity': '1 standard',
  'terrain': 'grasslands, mountains',
  'surface_water': '40',
  'population': '2000000000',
  'residents': [5, 68, 81],
  'films': [1, 6],
  'url': 'https://swapi.dev/api/planets/2/'}]

In [42]:
print(*rq.get('https://swapi.dev/api/films/1').json().keys(), sep='\n')

title
episode_id
opening_crawl
director
producer
release_date
characters
planets
starships
vehicles
species
created
edited
url


In [46]:
def get_film(id):
    base_url = 'https://swapi.dev/api/films'
    # get the url
    response = rq.get(f'{base_url}/{id}')

    # success
    if response.status_code == 200:
        content = response.json()
    elif response.status_code == 404:
        print(f'{base_url}/{id} not found!')
        return None
    
    for field in ['characters', 'planets', 'starships', 'vehicles', 'species']:
        # if the field is not empty
        if content[field]:
            id_values = []
            for link in content[field]:
                # parse the id value in the link    
                id_values.append(int(link.split('/')[-2]))
            content[field] = id_values
             
    # remove created and edited fields
    try:
        del(content['created'])
        del(content['edited'])
    except:
        pass
    
    return content

In [47]:
# a list of dictionaries with the character info
films_list = []
id = 1
while True:
    # the character page with id 17 doesn't exist!
    # if id == 17:
    #     id += 1
    #     pass
    film = get_film(id)
    if film:
        films_list.append(film)
        id += 1
    else:
        print(f'\nInformation from {id-1} planets has been extracted.')
        break

https://swapi.dev/api/films/7 not found!

Information from 6 planets has been extracted.


In [19]:
for index, value in enumerate(characters_list):
    if 'detail' in value.keys():
        print(f'Details found in character with id: {index + 1}')

Details found in character with id: 17


Remove character with id 17. It's a valid url!

In [27]:
del characters_list[16]

In [28]:
df = pd.DataFrame(characters_list)

In [30]:
df.head()

,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,films,species,vehicles,starships,url
0,Luke Skywalker,172,77,blond,fair,blue,19BBY,male,1,"[1, 2, 3, 6]",human,"[14, 30]","[12, 22]",https://swapi.dev/api/people/1/
1,C-3PO,167,75,n/a,gold,yellow,112BBY,n/a,1,"[1, 2, 3, 4, 5, 6]",[2],[],[],https://swapi.dev/api/people/2/
2,R2-D2,96,32,n/a,"white, blue",red,33BBY,n/a,8,"[1, 2, 3, 4, 5, 6]",[2],[],[],https://swapi.dev/api/people/3/
3,Darth Vader,202,136,none,white,yellow,41.9BBY,male,1,"[1, 2, 3, 6]",human,[],[13],https://swapi.dev/api/people/4/
4,Leia Organa,150,49,brown,light,brown,19BBY,female,2,"[1, 2, 3, 6]",human,[30],[],https://swapi.dev/api/people/5/


In [31]:
df.shape

(81, 14)

In [32]:
df.to_csv('sw_characters.csv')